# Group Project 1130

## *AirBnb DataSet Problem*

## Goals (3 min)

To understand and work with the AirBnb datasets to solve our business problem 

## Introduction (5 min)

**Business Context.** We are a company looking to expand our ventures based off tourism.

**Business Problem.** The main Task is  **wrangle datasets related to AirBnb and create a critera to figure out the areas to build in**.

**Analytical Context.** Text data is highly unstructured, and often requires pre-processing before we can gather any business insights from it. We will be leveraging tools from **natural language processing (NLP)** in order to help us process this data and generate new features that can be used for analytics or model building.

The case will proceed as follows: 
1. We will extract the files based off what they are
2. We will clean the data 
3. Create bar graph and determine average age in each major zipcode 
4. Same as above but with economic demos 
5. Average Airbnb review per zip code /posyiieve airbnb reviews if they're text
6. Average size of Airbnb per zip code 
7. Average cost of Airbnb per zip code 
8. Airbnb date availability
9. Using all these graphs and our critera we will decide where to build

In [1]:
import nltk # imports the natural language toolkit
nltk.download('punkt')
nltk.download('stopwords')
import pandas as pd
import numpy  as np
import string
import zipfile
import plotly
import tarfile
import gzip
from nltk.stem import PorterStemmer 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abdul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abdul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# LOADING THE DATASETS
demo = pd.read_csv('demographics.csv')
econ = pd.read_csv('econ_state.csv')
listings = pd.read_csv('listings.csv')

realestate = pd.read_csv('real_estate.csv.gz', compression='gzip')

venues = pd.read_csv('venues.csv.gz', compression='gzip')

zf = zipfile.ZipFile('calendar.csv.zip') 
calendar = pd.read_csv(zf.open('calendar.csv'))
ziplook = pd.read_csv('zipcodesForLookup.csv')
lists = pd.read_csv('listings.csv')

In [3]:
demo.head()

,zipcode,5_years_or_less,5-9_years,10-14_years,15-19_years,20-24_years,25-34_years,35-44_years,45-54_years,55-59_years,...,"$10,000-$14,999","$15,000-$24,999","$25,000-$34,999","$35,000-$49,999","$50,000-$64,999","$65,000-$74,999","$75,000-$99,999","$100,000_or_more",median_household_income,mean_household_income
0,601,17982,1006,1080,1342,1352,1321,2253,2149,2434,...,48.1,12,12.8,8.6,8.7,6.2,1.4,16.3,10816,20349
1,602,40260,2006,2440,2421,2953,2865,5124,5139,5947,...,31.4,16.3,17.9,12.2,10.6,7.7,2.9,21.2,16079,23282
2,603,52408,2664,3177,3351,3685,3585,6473,6775,6678,...,31,14.9,17.5,11.7,10.8,8.7,2.4,21.9,16804,26820
3,606,6331,347,331,461,474,469,707,933,776,...,45.3,10.2,20,11.7,11,1.8,0,12.8,12512,15730
4,610,28328,1438,1490,2044,2122,1985,3358,3778,3858,...,26.9,14.8,23.7,15.2,9.3,7.5,1.6,18.4,17475,23360


In [4]:
econ.head()

,state,2005Q1_gdp,2005Q2_gdp,2005Q3_gdp,2005Q4_gdp,2006Q1_gdp,2006Q2_gdp,2006Q3_gdp,2006Q4_gdp,2007Q1_gdp,...,2016/03_ur,2016/04_ur,2016/05_ur,2016/06_ur,2016/07_ur,2016/08_ur,2016/09_ur,2016/10_ur,2016/11_ur,2016/12_ur
0,AL,153332,155940,157437,160293,161934,163974,165470,166495,166821,...,6.6,6.6,6.6,6.7,6.7,6.7,6.6,6.6,6.6,6.6
1,AK,37517,38907,40691,43138,42872,44653,45349,45840,46658,...,5.9,5.8,5.8,5.8,5.8,5.9,6.0,6.1,6.2,6.3
2,AZ,218206,224496,231629,235099,241787,244659,250886,256505,258078,...,4.1,4.1,4.1,4.1,4.0,4.0,4.0,4.0,4.0,3.9
3,AR,88446,89264,90515,93050,93413,95259,95481,95203,94289,...,5.5,5.4,5.3,5.3,5.2,5.1,5.1,5.0,5.0,5.0
4,CA,1722091,1747827,1787427,1809426,1857944,1865835,1886549,1907754,1915172,...,5.6,5.5,5.5,5.5,5.4,5.4,5.3,5.3,5.3,5.2


In [5]:
realestate.head()

,type,zipcode,city,state,metro,county,size_rank,1996-04,1996-05,1996-06,...,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06
0,ZHVI,10025,New York,NY,New York,New York,1,NaN,NaN,NaN,...,1137500,1137700,1152700,1156000.0,1140200,1130000,1131900,1149600,1198400,1247000
1,ZHVI,60657,Chicago,IL,Chicago,Cook,2,146700.0,146500.0,146300.0,...,318200,318100,318800,320200.0,320800,322000,323800,326100,327800,329100
2,ZHVI,60614,Chicago,IL,Chicago,Cook,3,198000.0,195500.0,194200.0,...,401900,406000,414100,417800.0,417400,418400,414100,404100,406400,415500
3,ZHVI,79936,El Paso,TX,El Paso,El Paso,4,70800.0,71000.0,71000.0,...,113800,113900,114100,114500.0,114900,115000,114700,114700,114800,114700
4,ZHVI,60640,Chicago,IL,Chicago,Cook,5,102300.0,101300.0,100700.0,...,198800,199200,200100,201500.0,203000,205100,206700,206500,206200,206700


In [6]:
calendar.head()

,listing_id,date,available,price,metro_area
0,2515,2018-03-05,t,69.0,NYC
1,2515,2018-03-04,t,69.0,NYC
2,2515,2018-03-03,t,69.0,NYC
3,2515,2018-03-02,t,69.0,NYC
4,2515,2018-03-01,t,69.0,NYC


In [7]:
venues.head()

,city,id,latitude,longitude,name,rating,types
0,new york city,b1a0d113cb17d1d85f0e12700dd71f36bddedc54,40.601540,-73.729636,A Bacon Yacht Charter,NaN,"['restaurant', 'food', 'point_of_interest', 'e..."
1,new york city,8799bb8d8ac7c2e1933f92ac1ef0f69a5e0b37c1,40.608921,-73.728256,Mezzanote,NaN,"['restaurant', 'food', 'point_of_interest', 'e..."
2,new york city,f2390680116af4d62e0da6f0432d33e94b9cb0e6,40.616978,-73.730349,Prime Bistro,4.0,"['bar', 'restaurant', 'food', 'point_of_intere..."
3,new york city,e9294eb56025e8eb29b937dc0aca29fd3059b9ab,40.617318,-73.730637,Rita's Ice,4.6,"['restaurant', 'food', 'point_of_interest', 'e..."
4,new york city,5c7b72c7bf9e7bc4f94baf09db1252088087f65d,40.618371,-73.728178,Cho-Sen Island,4.4,"['restaurant', 'food', 'point_of_interest', 'e..."


In [8]:
lists.head()

,accommodates,amenities,availability_30,bathrooms,bed_type,bedrooms,beds,cancellation_policy,city,has_availability,...,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,room_type,state,weekly_price,zipcode
0,2.0,"{""Cable TV"",""Wireless Internet"",""Air condition...",24,1.0,Real Bed,1.0,1.0,moderate,sunnysidebronx,NaN,...,10.0,10.0,10.0,10.0,100.0,10.0,Private room,NY,NaN,10464
1,4.0,"{TV,Internet,""Wireless Internet"",""Air conditio...",30,1.0,Real Bed,1.0,1.0,flexible,sunnysidebronx,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Private room,NY,NaN,10464
2,4.0,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",30,3.0,Real Bed,3.0,3.0,strict,sunnysidebronx,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Entire home/apt,NY,NaN,10464
3,3.0,"{TV,Internet,""Wireless Internet"",""Air conditio...",8,1.0,Real Bed,1.0,1.0,strict,long island city,NaN,...,10.0,10.0,10.0,10.0,93.0,10.0,Entire home/apt,NY,775.0,10464
4,4.0,"{Internet,""Wireless Internet"",""Air conditionin...",17,1.0,Real Bed,1.0,1.0,moderate,sunnysidebronx,NaN,...,10.0,10.0,10.0,10.0,97.0,10.0,Private room,NY,350.0,10464


### Exercise 1: (4 min)

Clean the data for NYC 

In [9]:
lists = lists.drop(['accommodates', 'amenities', 'availability_30','bathrooms','bed_type','bedrooms','beds'], axis=1) 
lists = lists.drop(['cancellation_policy', 'has_availability', 'instant_bookable','latitude','longitude'], axis=1) 
lists = lists.drop(['property_type', 'room_type'], axis=1) 
lists['avg review'] = lists[['review_scores_checkin', 'review_scores_cleanliness','review_scores_communication','review_scores_location','review_scores_rating','review_scores_value']].mean(axis=1)
lists = lists.drop(['review_scores_checkin', 'review_scores_cleanliness','review_scores_communication','review_scores_location','review_scores_rating','review_scores_value'], axis=1) 
lists = lists[lists.state == 'NY']
lists[lists.columns[~lists.isnull().any()]]
lists = lists[lists.metropolitan == 'NYC']
lists[lists.columns[~lists.isnull().any()]]
lists = lists.drop(['host_id', 'metropolitan','state'], axis=1) 
lists

,city,id,name,price,weekly_price,zipcode,avg review
0,sunnysidebronx,7949480,City Island Sanctuary relaxing BR & Bath w Par...,99.0,NaN,10464,25.000000
1,sunnysidebronx,16042478,WATERFRONT STUDIO APARTMENT,200.0,NaN,10464,NaN
2,sunnysidebronx,1886820,Quaint City Island Community.,300.0,NaN,10464,NaN
3,long island city,6627449,Large 1 BDRM in Great location,125.0,775.0,10464,23.833333
4,sunnysidebronx,5557381,Quaint City Island Home,69.0,350.0,10464,24.500000
...,...,...,...,...,...,...,...
40735,yonkers,18197581,Cozy room with parking lot included,70.0,NaN,10705,NaN
40736,new york,3235285,Duplex Garden apt off Central Park!,949.0,6200.0,10024,25.000000
40737,new york,18324921,Yacht phish shows Madison sq garden,600.0,NaN,10004,NaN
40738,new york,18035489,In the heart of tourist attractions,129.0,NaN,10029,NaN


In [10]:
# Cleaning venues, only get NYC drops NA values
venuescl = venues[venues.city == 'new york city']
venuescl[venuescl.columns[~venues.isnull().any()]]
venuescl = venuescl.dropna(how='any',axis=0) 
venuescl.head()


,city,id,latitude,longitude,name,rating,types
2,new york city,f2390680116af4d62e0da6f0432d33e94b9cb0e6,40.616978,-73.730349,Prime Bistro,4.0,"['bar', 'restaurant', 'food', 'point_of_intere..."
3,new york city,e9294eb56025e8eb29b937dc0aca29fd3059b9ab,40.617318,-73.730637,Rita's Ice,4.6,"['restaurant', 'food', 'point_of_interest', 'e..."
4,new york city,5c7b72c7bf9e7bc4f94baf09db1252088087f65d,40.618371,-73.728178,Cho-Sen Island,4.4,"['restaurant', 'food', 'point_of_interest', 'e..."
5,new york city,0b99220b44ee0d45d28f44e95d08da112f6e2ca7,40.618126,-73.728679,Sunflower Cafe- Lawrence,4.2,"['cafe', 'restaurant', 'food', 'point_of_inter..."
6,new york city,97fdbf124eeb91d70d7c3f710ca52c997d618bb9,40.617592,-73.729390,Sushi Tokyo Lawrence,4.0,"['restaurant', 'food', 'point_of_interest', 'e..."


In [11]:
# Cleaning Calender, drops NAS and gets only avaliable airbnbs
calendar_aval = calendar[calendar.available == 't']
calendar_aval[calendar_aval.columns[~calendar_aval.isnull().any()]]
calendar_aval = calendar_aval[calendar_aval.metro_area == 'NYC']
calendar_aval[calendar_aval.columns[~calendar_aval.isnull().any()]]
calendar_aval.head()

,listing_id,date,available,price,metro_area
0,2515,2018-03-05,t,69.0,NYC
1,2515,2018-03-04,t,69.0,NYC
2,2515,2018-03-03,t,69.0,NYC
3,2515,2018-03-02,t,69.0,NYC
4,2515,2018-03-01,t,69.0,NYC


In [12]:
# Cleaning real estate, gets only 2016-08 onwards
realcleaned = realestate.filter(["type","zipcode","city","state","metro","county","size_rank","2016-08","2016-09","2016-10","2016-11","2016-12","2017-01","2017-02","2017-03","2017-04","2017-05","2017-06"], axis=1)
realcleaned = realcleaned[realcleaned.city == 'New York']
realcleaned[realcleaned.columns[~realcleaned.isnull().any()]]
realcleaned.head()

,type,zipcode,city,state,metro,county,size_rank,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06
0,ZHVI,10025,New York,NY,New York,New York,1,1132500,1137500,1137700,1152700,1156000.0,1140200,1130000,1131900,1149600,1198400,1247000
8,ZHVI,11226,New York,NY,New York,Kings,9,572800,583600,594800,605200,612100.0,612800,616900,628900,644200,653500,658700
10,ZHVI,10016,New York,NY,New York,New York,11,917000,934800,946000,949200,950000.0,951800,960100,972200,986900,1021200,1061200
17,ZHVI,10128,New York,NY,New York,New York,18,1077000,1074900,1076400,1090300,1111300.0,1133100,1154700,1189800,1241000,1288200,1318300
23,ZHVI,10462,New York,NY,New York,Bronx,24,118200,120600,121600,121500,121000.0,121100,123200,126200,128700,131900,135400


In [13]:
#Econ doesnt need cleaning
democlean = demo.filter(["zipcode","20-24_years","25-34_years","35-44_years","35,000","50,000","65,000","75,000","households","$9,999_or_less","$10,000-$14,999","$15,000-$24,999","$25,000-$34,999","$35,000-$49,999","$50,000-$64,999","$65,000-$74,999","$75,000-$99,999","$100,000_or_more","median_household_income","mean_household_income"], axis=1)
democlean = demo.dropna(how='any',axis=0) 
democlean.head()

,zipcode,5_years_or_less,5-9_years,10-14_years,15-19_years,20-24_years,25-34_years,35-44_years,45-54_years,55-59_years,...,"$10,000-$14,999","$15,000-$24,999","$25,000-$34,999","$35,000-$49,999","$50,000-$64,999","$65,000-$74,999","$75,000-$99,999","$100,000_or_more",median_household_income,mean_household_income
0,601,17982,1006,1080,1342,1352,1321,2253,2149,2434,...,48.1,12,12.8,8.6,8.7,6.2,1.4,16.3,10816,20349
1,602,40260,2006,2440,2421,2953,2865,5124,5139,5947,...,31.4,16.3,17.9,12.2,10.6,7.7,2.9,21.2,16079,23282
2,603,52408,2664,3177,3351,3685,3585,6473,6775,6678,...,31,14.9,17.5,11.7,10.8,8.7,2.4,21.9,16804,26820
3,606,6331,347,331,461,474,469,707,933,776,...,45.3,10.2,20,11.7,11,1.8,0,12.8,12512,15730
4,610,28328,1438,1490,2044,2122,1985,3358,3778,3858,...,26.9,14.8,23.7,15.2,9.3,7.5,1.6,18.4,17475,23360


### Cleaning data to work with it

Here we work on the data to get it so we can analyze it further by adding zipcodes and doing zipcode lookup

In [14]:
zipdict = ziplook.set_index('Column1').T.to_dict('list')
zipdict

{10001: ['Manhattan'],
 10002: ['Manhattan'],
 10003: ['Manhattan'],
 10004: ['Manhattan'],
 10005: ['Manhattan'],
 10006: ['Manhattan'],
 10007: ['Manhattan'],
 10009: ['Manhattan'],
 10010: ['Manhattan'],
 10011: ['Manhattan'],
 10012: ['Manhattan'],
 10013: ['Manhattan'],
 10014: ['Manhattan'],
 10015: ['Manhattan'],
 10016: ['Manhattan'],
 10017: ['Manhattan'],
 10018: ['Manhattan'],
 10019: ['Manhattan'],
 10020: ['Manhattan'],
 10021: ['Manhattan'],
 10022: ['Manhattan'],
 10023: ['Manhattan'],
 10024: ['Manhattan'],
 10025: ['Manhattan'],
 10026: ['Manhattan'],
 10027: ['Manhattan'],
 10028: ['Manhattan'],
 10029: ['Manhattan'],
 10030: ['Manhattan'],
 10031: ['Manhattan'],
 10032: ['Manhattan'],
 10033: ['Manhattan'],
 10034: ['Manhattan'],
 10035: ['Manhattan'],
 10036: ['Manhattan'],
 10037: ['Manhattan'],
 10038: ['Manhattan'],
 10039: ['Manhattan'],
 10040: ['Manhattan'],
 10041: ['Manhattan'],
 10044: ['Manhattan'],
 10045: ['Manhattan'],
 10048: ['Manhattan'],
 10055: ['M

In [15]:
realcleaned['borough'] = realcleaned['zipcode'].map(zipdict)
realcleaned.head()

,type,zipcode,city,state,metro,county,size_rank,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,borough
0,ZHVI,10025,New York,NY,New York,New York,1,1132500,1137500,1137700,1152700,1156000.0,1140200,1130000,1131900,1149600,1198400,1247000,[Manhattan]
8,ZHVI,11226,New York,NY,New York,Kings,9,572800,583600,594800,605200,612100.0,612800,616900,628900,644200,653500,658700,[Brooklyn]
10,ZHVI,10016,New York,NY,New York,New York,11,917000,934800,946000,949200,950000.0,951800,960100,972200,986900,1021200,1061200,[Manhattan]
17,ZHVI,10128,New York,NY,New York,New York,18,1077000,1074900,1076400,1090300,1111300.0,1133100,1154700,1189800,1241000,1288200,1318300,[Manhattan]
23,ZHVI,10462,New York,NY,New York,Bronx,24,118200,120600,121600,121500,121000.0,121100,123200,126200,128700,131900,135400,[Bronx]


In [16]:
# drop city, state, metro, county, size_rank columns
realcleaned  = realcleaned.drop(['city', 'state', 'metro', 'county', 'size_rank'], axis=1) 
#remove square brackets from start and end of borough column
realcleaned['borough'] = realcleaned['borough'].str[0]
realcleaned.head() 

,type,zipcode,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,borough
0,ZHVI,10025,1132500,1137500,1137700,1152700,1156000.0,1140200,1130000,1131900,1149600,1198400,1247000,Manhattan
8,ZHVI,11226,572800,583600,594800,605200,612100.0,612800,616900,628900,644200,653500,658700,Brooklyn
10,ZHVI,10016,917000,934800,946000,949200,950000.0,951800,960100,972200,986900,1021200,1061200,Manhattan
17,ZHVI,10128,1077000,1074900,1076400,1090300,1111300.0,1133100,1154700,1189800,1241000,1288200,1318300,Manhattan
23,ZHVI,10462,118200,120600,121600,121500,121000.0,121100,123200,126200,128700,131900,135400,Bronx


In [18]:
democlean['borough'] = democlean['zipcode'].map(zipdict)
democlean = democlean.dropna(how='any',axis=0) 
democlean['borough'] = democlean['borough'].str[0]

democlean.head()

,zipcode,5_years_or_less,5-9_years,10-14_years,15-19_years,20-24_years,25-34_years,35-44_years,45-54_years,55-59_years,...,"$15,000-$24,999","$25,000-$34,999","$35,000-$49,999","$50,000-$64,999","$65,000-$74,999","$75,000-$99,999","$100,000_or_more",median_household_income,mean_household_income,borough
2558,10001,23537,562,1089,900,1053,2785,4970,3905,2795,...,4.1,9.3,5.1,9.2,8.5,9.8,27.5,86801,158183,Manhattan
2559,10002,80736,3079,3171,3638,3922,5338,13938,12055,11182,...,12.6,14,8.8,10,14.2,7.1,31.3,33726,61946,Manhattan
2560,10003,57112,1545,911,764,7308,5714,15565,7843,5718,...,1.9,7,4.9,6.4,13,11.3,30.7,98151,161692,Manhattan
2561,10004,3221,126,101,161,227,255,857,572,387,...,0.3,1.7,6.5,5.1,7.1,14.8,27,119691,177262,Manhattan
2562,10005,8131,299,250,140,85,1101,3690,1696,631,...,0.8,4.5,1.9,1.4,9,9.9,20.3,124194,176424,Manhattan


In [19]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.point import Point
import geopy

def get_zip_code(x):
    geolocator = geopy.Nominatim(user_agent="check_1")
    location = geolocator.reverse("{}, {}".format(x['latitude'],x['longitude']))
    return location.raw['address']['postcode']
venu = venuescl.sort_values(by=['rating'], ascending=False)
venu = venu.head(50)
venu['zipcode'] = venu.apply(lambda x: get_zip_code(x), axis = 1)
venu.head()


,city,id,latitude,longitude,name,rating,types,zipcode
48476,new york city,5161e3e3db931a8574e44c063f4ed8647375e8a9,40.669737,-73.842051,Boulevard Discount,5.0,"['home_goods_store', 'store', 'point_of_intere...",11417
59758,new york city,0bce4e42ae30780b6da1c24edf56a2070a041e7e,40.746251,-73.913639,Discount Variety & Grocery.,5.0,"['grocery_or_supermarket', 'food', 'store', 'p...",11377
59855,new york city,b113763e103d2d45836b1f6c92b8a4d1d39867fb,40.767347,-73.911814,"TANJAWI, Hallal Food Emporium Inc",5.0,"['food', 'store', 'point_of_interest', 'establ...",11103
59847,new york city,5a005af8307213bfcf140bbe3536770010c20679,40.766207,-73.913381,Watany Food Market,5.0,"['grocery_or_supermarket', 'food', 'store', 'p...",11103
59840,new york city,ce4779dd89bb0912874f1f8e7795adb94bc54474,40.766203,-73.913372,Steinway Pharmacy,5.0,"['pharmacy', 'health', 'store', 'point_of_inte...",11103


In [20]:
venu['zipcode']  = venu['zipcode'].astype(int)
venu['borough'] = venu['zipcode'].map(zipdict)
venu.head()

,city,id,latitude,longitude,name,rating,types,zipcode,borough
48476,new york city,5161e3e3db931a8574e44c063f4ed8647375e8a9,40.669737,-73.842051,Boulevard Discount,5.0,"['home_goods_store', 'store', 'point_of_intere...",11417,[Queens]
59758,new york city,0bce4e42ae30780b6da1c24edf56a2070a041e7e,40.746251,-73.913639,Discount Variety & Grocery.,5.0,"['grocery_or_supermarket', 'food', 'store', 'p...",11377,[Queens]
59855,new york city,b113763e103d2d45836b1f6c92b8a4d1d39867fb,40.767347,-73.911814,"TANJAWI, Hallal Food Emporium Inc",5.0,"['food', 'store', 'point_of_interest', 'establ...",11103,[Queens]
59847,new york city,5a005af8307213bfcf140bbe3536770010c20679,40.766207,-73.913381,Watany Food Market,5.0,"['grocery_or_supermarket', 'food', 'store', 'p...",11103,[Queens]
59840,new york city,ce4779dd89bb0912874f1f8e7795adb94bc54474,40.766203,-73.913372,Steinway Pharmacy,5.0,"['pharmacy', 'health', 'store', 'point_of_inte...",11103,[Queens]


In [21]:
# drop city, state, metro, county, size_rank columns
venu  = venu.drop(['id', 'latitude', 'longitude'], axis=1) 
#remove square brackets from start and end of borough column
venu['borough'] = venu['borough'].str[0]
venu.head()

,city,name,rating,types,zipcode,borough
48476,new york city,Boulevard Discount,5.0,"['home_goods_store', 'store', 'point_of_intere...",11417,Queens
59758,new york city,Discount Variety & Grocery.,5.0,"['grocery_or_supermarket', 'food', 'store', 'p...",11377,Queens
59855,new york city,"TANJAWI, Hallal Food Emporium Inc",5.0,"['food', 'store', 'point_of_interest', 'establ...",11103,Queens
59847,new york city,Watany Food Market,5.0,"['grocery_or_supermarket', 'food', 'store', 'p...",11103,Queens
59840,new york city,Steinway Pharmacy,5.0,"['pharmacy', 'health', 'store', 'point_of_inte...",11103,Queens


In [31]:
democlean['mean_household_income']  = pd.to_numeric(democlean['mean_household_income'] , errors='coerce')

,zipcode,5_years_or_less,5-9_years,10-14_years,15-19_years,20-24_years,25-34_years,35-44_years,45-54_years,55-59_years,60-64_years,65-74_years,75-84_years,85_years_or_more,households,"$9,999_or_less"
mean_household_income,,,,,,,,,,,,,,,,
53051.840000,10463.000000,57202.080000,4342.480000,4015.320000,3977.320000,4169.800000,4844.560000,8631.880000,7373.520000,7617.760000,3213.800000,2658.760000,3565.000000,1956.720000,835.160000,19534.760000
72558.297297,11220.297297,70142.135135,5227.270270,4450.918919,4177.945946,4147.702703,5228.918919,12469.972973,9684.486486,8729.729730,4051.729730,3615.000000,4612.540541,2544.918919,1201.000000,25183.405405
80195.033898,11378.111111,36518.000000,2275.555556,2021.396825,1946.206349,2020.714286,2576.222222,5960.587302,5248.365079,5196.968254,2398.412698,2029.904762,2654.444444,1495.698413,693.523810,12332.698413
87952.750000,10307.076923,36344.692308,2142.076923,2244.230769,2339.923077,2328.846154,2498.307692,4678.000000,4823.846154,5400.076923,2549.000000,2233.538462,2945.153846,1486.769231,674.923077,12752.615385
146706.093023,10083.940299,23344.940299,1184.507463,880.238806,845.985075,1051.492537,1822.820896,5286.089552,3429.179104,2968.447761,1336.164179,1298.686567,1781.701493,983.611940,476.014925,10687.537313


In [43]:
d = democlean.groupby(['borough']).mean()
d = d.drop(['20-24_years', '5_years_or_less', '5-9_years','10-14_years','15-19_years','25-34_years','35-44_years','45-54_years'], axis=1) 
d = d.drop(['55-59_years', '60-64_years', '65-74_years','75-84_years','85_years_or_more','zipcode','$9,999_or_less',], axis=1) 
d

,households,mean_household_income
borough,,
Bronx,835.160000,53051.840000
Brooklyn,1201.000000,72558.297297
Manhattan,476.014925,146706.093023
Queens,693.523810,80195.033898
Staten,674.923077,87952.750000


### Exercise 2: (5 min)

Conduct an exploratory analysis of the sizes of reviews: find the shortest and longest reviews, then plot a histogram showing the distribution of review lengths.

**Answer.** One possible solution is shown below:

## Text visualization with word clouds (15 min)

Just like visualization is crucial for standard CSV data, it is also important for text data. But text doesn't lend itself to histograms or scatterplots the way that numerical or even categorical data do. In such cases, **word clouds** are a common and <font color="orange">sometimes</font> useful tool.

**Note:** While wordclouds can be a useful way of quickly gaining high level insights into raw textual data, they are also limited. In some ways, they can be seen as the pie charts of NLP: often used, but also often hated. [Some](https://www.niemanlab.org/2011/10/word-clouds-considered-harmful/) [people](https://towardsdatascience.com/can-we-please-stop-using-word-clouds-eca2bbda7b9d) would prefer if they didn't exist at all. If used in the correct way, however, they definitely deserve their place in a data scientist's toolbelt.

The main problem with word clouds is that they are difficult to interpret in a standard way. The layout algorithm has some randomness involved and although more common words are shown more prominently, it's not possible to look at a word cloud and know which words are the most important, or how much more important these are than other words. Colours and rotation are also used randomly, making some words (e.g the ones in bright colours, positioned closer to the centre, with horizontal rotation) seem more important when in fact they are no more important than other words which were randomly assigned a less noticeable combination of color, rotation, and position.

### Exercise 3: (7 min)

Write a function `word_cloud_rating(data, star_value)` that constructs a word cloud from the subset of `data` that exhibit a certain `star_value`. Visualize the results of this function for 1-star reviews.

**Answer.** One possible solution is given below:

In [ ]:
## Setting the resolution for better clarity 
from pylab import rcParams
rcParams['figure.figsize'] = 30, 60

def word_cloud_rating(data,star_value):
    
    data_filtered = data[data.stars == star_value] #filtering according to the star value
    Reviews = data_filtered.text

    Reviews_text = ' '.join(Reviews.values) #joining all the words together

    # Creating a word cloud object
    wordcloud = WordCloud(max_font_size=100, max_words=100, background_color="white",\
                          scale = 10,width=800, height=400).generate(Reviews_text)


    # Plotting the generated word cloud
    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()

In [ ]:
word_cloud_rating(data, 1)

### Exercise 4: (5 min)

The word "good" seems to appear quite frequently in the negative reviews. Investigate why that is and come up with a reasonable explanation.

 **Answer.** Let's look at the first 5 reviews or so with 1-star ratings to see if there are any discernible patterns:

In [ ]:
reviews_containing_good = [each for each in data[data.stars == 1].text if 'good' in each]
for review in reviews_containing_good[:20]:
    good_index = review.find("good")
    print(review[good_index-20:good_index+20].replace("\n", ""))

Reading each of the reviews, it is clear that "good" is often mentioned in a context like "no good place to sit" or "sound good". This indicates that in the world of text we cannot go by single words (also called **1-grams**) alone. The context of the sentence or the surrounding words at least are very much necessary to understand the sentiment of a sentence.

## n-grams (25 min)

Since 1-grams are insufficient to understand the significance of certain words in our text, it is natural to consider blocks of words, or **n-grams**.

The simplest version of the n-gram model, for $n > 1$, is the **bigram** model, which looks at pairs of consecutive words. For example, the sentence "The quick brown fox jumps over the lazy dog" would have tokens "the quick", "quick brown",..., "lazy dog". The following image explains this concept:

<img src="ngrams.png" alt="ngrams" width="500"/>

This has obvious advantages and disadvantages over looking at words individually:

1. This retains the structure of the overall document, and
2. It paves the way for analyzing words in contex; however,
3. The dimension is vastly larger

In practice, this last challenge can be truly daunting. As an example, *War and Peace* has 3 million characters, which translates to several hundred thousand 1-grams (words). If you consider that the set of all possible bigrams can be as large as the square of the number of 1-grams, this gets us to a hundred billion possible bigrams! If classical ML techniques are not suitable for training on 3 million characters, how can they possibly deal with a hundred billion dimensions?

For this reason, it is often prudent to start by extracting as much value out of 1-grams as possible, before working our way up to more complex structures.

In this section we also start to look again at our main application: calculating some "interesting" features of our corpus of reviews.

When thinking about word analysis, the main topic of interest is finding an *efficient* and *low-dimensional* representation in order to facilitate document visualization and larger-scale analyses. We discuss two broad categories of word representations:

1. `Count-based representations`: word-word and word-document matrices.
2. `Word embeddings`: spectral embedding, UMAP, word2vec, GloVe, and many many more.

These are often used to assist with downstream tasks such as clustering, ranking and labeling, which will be briefly discussed in a future case. Word embeddings in particular have become something of a posterchild. These, combined with neural networks (which will also be discussed in a future case!), have led to many of the recent headline improvements in NLP tasks.

### Count-based representations (20 min)

n-grams fall under a broader category of techniques otherwise known as [**count-based representations**](https://scikit-learn.org/stable/modules/feature_extraction.html#common-vectorizer-usage). These are techniques to analyze documents by indicating how frequently certain types of structures occur throughout.

Let's start with 1-grams (words). The simplest type of information would be whether a particular word occurs in particular documents. This leads to **word-document co-occurrence matrices**, where the $(W, X)$ entry of the word-document matrix is set to 1 if word $W$ occurs in document $X$, and 0 otherwise.

There are many variants of this. In lieu of the fact that we are looking for count-based representations of our documents, one natural variable is the following: the $(W, X)$ entry of the word-document matrix equals the number of times that word $W$ occurs in document $X$, rather than merely being a binary variable.

Let's create a word-document co-occurrence matrix for our set of reviews:

In [ ]:
# The following code creates a word-document matrix.
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
X = vec.fit_transform(AllReviews)
df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
df.head()

### Exercise 5: (5 min)

Find all the high-frequency (top 1%) and low-frequency (bottom 1%) words in the reviews overall. (Hint: import the `Counter()` function from the `collections` class.)

**Answer.** One possible solution is shown below:

In [ ]:
from collections import Counter

all_reviews_text = ' '.join(AllReviews)
tokenized_words = nltk.word_tokenize(all_reviews_text)
word_freq = Counter(tokenized_words)

len(word_freq)

In [ ]:
## Therefore, top 1% is ~463 words
word_freq.most_common(463)

In [ ]:
## Similarly, bottom 1% is ~463 words
word_freq.most_common()[-463:-1]

Let's do the same for bigrams. Here is the code to get the set of bigrams for the first 5 reviews:

In [ ]:
from nltk.util import ngrams

first_5_revs = data.text[0:5]
word_tokens = nltk.word_tokenize(''.join(first_5_revs))
list(ngrams(word_tokens, 2)) #ngrams(word_tokens,n) gives the n-grams.

### Exercise 6: (12 min)

Write a function called `top_k_ngrams(word_tokens, n, k)` for printing out the top $k$ n-grams. Use this function to get the top 10 1-grams, 2-grams, and 3-grams from the first 1000 reviews in our dataset.

**Answer.** One possible solution is shown below:

In [ ]:
from nltk.util import ngrams

def top_k_ngrams(word_tokens,n,k):
    
    ## Getting them as n-grams
    n_gram_list = list(ngrams(word_tokens, n))

    ### Getting each n-gram as a separate string
    n_gram_strings = [' '.join(each) for each in n_gram_list]
    
    n_gram_counter = Counter(n_gram_strings)
    most_common_k = n_gram_counter.most_common(k)
    print(most_common_k)
    
   # x_pos = [k for k,v in most_common_k]
   # y_pos = [v for k,v in most_common_k]
    
   # plt.bar(x_pos, y_pos,align='center')

In [ ]:
### Getting a single string
top_1000_reviews = data.text[0:1000]
all_reviews_text = ' '.join(top_1000_reviews)

## Splitting them into tokens
word_tokens = nltk.word_tokenize(all_reviews_text)

## Calling the function for top k
top_k_ngrams(word_tokens, 1, 10)

In [ ]:
top_k_ngrams(word_tokens, 2, 10)

In [ ]:
top_k_ngrams(word_tokens, 3, 10)

## Stop words (20 min)

You may have noticed a pattern in the types of words that show up in the top 10 1-grams, 2-grams, and 3-grams. In particular, these are common words that appear in every sentence of the English language: pronoums like "I", prepositions like "but", "of", "and", articles like "the", etc. These very common words are usually uninformative, and their very large occurrence values can distort the results of many NLP algorithms.

For this reason, it is common to pre-process text by removing words that you have a reason to believe are uninformative; these words are called [**stop words**](https://en.wikipedia.org/wiki/Stop_words). Usually, it suffices to simply treat extremely common words as stop words. However, for specific types of applications it might make sense to use other stop words; e.g. the word "burger" when analyzing reviews of burger chains.

(Note that stop words are often removed by default as a cleaning step in all NLP tasks. However, sometimes they can be useful. For example in authorship attribution (automatically detecting who wrote a specific piece of text by their 'writing style'), stop words can be one of the most useful features, as they appear in nearly all texts, and yet each author uses them in slightly different ways.)

The `nltk` library has a standard list of stopwords, which you can download by writing `nltk.download(“stopwords”)`. We can then load the stopwords package from the nltk.corpus and use it to load the stop words:

In [ ]:
print(stopwords.words("japanese"))

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
print(stopwords.words("english"))

You can get a list of all the Spanish stop words as well:

In [ ]:
print(stopwords.words("spanish"))

### Exercise 7: (15 min)

#### 7.1

Filter out all of the stop words in the first review of the Yelp review data and print out your answer. Additionally, print out (separately) the stopwords you found in this review.

**Answer.** One possible solution is given below:

In [ ]:
stop_words = set(stopwords.words("english"))
without_stop_words = []
stopword = []
sentence = data.text[0]
words = nltk.word_tokenize(sentence)
for word in words:
    if word in stop_words:
        stopword.append(word)
    else:
        without_stop_words.append(word)

print(stopword)
print()
print(without_stop_words)

#### 7.2

Modify the function `top_k_ngrams(word_tokens, n, k)` to remove stop words before determining the top n-grams.

**Answer.** Our recommended solution is shown below:

In [ ]:
# Removing the most basic stop words from the ntlk corpus and including only those
# words with character size above 2 so as to remove punctuations
# But, this could be extended to remove further high and low frequency stop words

from nltk.corpus import stopwords
import string
eng_stopwords = stopwords.words('english')

### Getting a single string
all_reviews_text = ' '.join(top_1000_reviews)

## Splitting them into tokens
word_tokens = nltk.word_tokenize(all_reviews_text)

## Removing the stopwords
word_tokens_clean = [each for each in word_tokens if each.lower() not in eng_stopwords and len(each.lower()) > 2]

## Calling the function for top k
top_k_ngrams(word_tokens_clean, 3, 10)

In [ ]:
top_k_ngrams(word_tokens_clean, 2, 10)

In some contexts, it is common to remove both very common and very *uncommon* words. The idea is that common words like "a" are almost never informative, while uncommon words like "syzygy" occur so infrequently in a corpus that many algorithms have a hard time processing them in a meaningful way. We will not deal with uncommon words today, but you should be aware that doing so improves the performance of several NLP techniques.

## Finding important words (30 min)

Up to this point, we have focused on techniques for transforming our data. We are now ready to start looking for some answers, so let's take a break from discussing techniques so we can explore our dataset and various ways to summarize it.

We begin by looking at the words and n-grams that are most common in positive and negative reviews. Note that in the following code, we don't reuse many of the pre-processing steps discussed at the start of the tutorial. This is because many of them are included as options in existing packages. In a serious project one would often customize this pre-processing to some degree, but we skip this in order to get some displayable results.

In [ ]:
# Following code grabbed from:
# https://towardsdatascience.com/a-complete-exploratory-data-analysis-and-visualization-for-text-data-29fb1b96fb6a
# we will use it in our context to create some visualizations.
def get_top_n_words(corpus, n=1,k=1):
    vec = CountVectorizer(ngram_range=(k,k),stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
# We start by getting a list of the most common words.

common_words = get_top_n_words(data['text'], 20,1)
for word, freq in common_words:
    print(word, freq)
df = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])
df.groupby('ReviewText').sum()['count'].sort_values(ascending=False).plot(
    kind='bar',  title='Top 20 words from all reviews')


### Exercise 8: (15 min)

#### 8.1

Divide the data into "good reviews" (i.e. `stars` rating was greater than 3) and "bad reviews" (i.e. `stars` rating was less than 3) and make a bar plot of the top 20 words in each case. Are these results different from above?

**Answer.** One possible solution is shown here:

In [ ]:
data.head()

In [ ]:
# We continue by splitting according to good/bad review scores, then grabbing again.

GoodInd = data['stars'] >3.1
GoodRev = data[GoodInd]
BadInd = data['stars'] <2.1
BadRev = data[BadInd]

common_words = get_top_n_words(GoodRev['text'], 20)
for word, freq in common_words:
    print(word, freq)
df = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])
df.groupby('ReviewText').sum()['count'].sort_values(ascending=False).plot(
    kind='bar', title='Top 20 words from good reviews')

In [ ]:
common_words = get_top_n_words(BadRev['text'], 20)
for word, freq in common_words:
    print(word, freq)
df = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])
df.groupby('ReviewText').sum()['count'].sort_values(ascending=False).plot(
    kind='bar', title='Top 20 words from bad reviews')

Well, that was pretty useless. The "good" words are mostly a mix of generic words like "place" and overtly positive words like "good" itself.

The problem here is that we are dealing with single words, which cannot convey much information out of context. The natural solution then is to deal with n-grams, so that we can get context-aware results like "good burger" or "good service" (in the positive reviews) or, as we saw, "good 45 minutes" (in the negative reviews).

#### 8.2

Use the `get_top_n_words()` function to find the top 20 bigrams and trigrams. Do the results seem useful?

**Answer.** One possible solution is shown here:

In [ ]:
# Get top bigrams and trigrams from bad reviews
common_words = get_top_n_words(BadRev['text'], 20,2)

for word, freq in common_words:
    print(word, freq)

df = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])
df.groupby('ReviewText').sum()['count'].sort_values(ascending=False).plot(
    kind='bar', title='Top 20 bigrams from bad reviews')

In [ ]:
# Get top bigrams and trigrams from bad reviews
common_words = get_top_n_words(GoodRev['text'], 20,2)

for word, freq in common_words:
    print(word, freq)

df = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])
df.groupby('ReviewText').sum()['count'].sort_values(ascending=False).plot(
    kind='bar', title='Top 20 bigrams from bad reviews')

In [ ]:
common_words = get_top_n_words(BadRev['text'], 20,3)
for word, freq in common_words:
    print(word, freq)
df = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])
df.groupby('ReviewText').sum()['count'].sort_values(ascending=False).plot(
    kind='bar', title='Top 20 trigrams from bad reviews')

In [ ]:
common_words = get_top_n_words(GoodRev['text'], 20,3)
for word, freq in common_words:
    print(word, freq)
df = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])
df.groupby('ReviewText').sum()['count'].sort_values(ascending=False).plot(
    kind='bar', title='Top 20 bigrams from good reviews')

In [ ]:
common_words = get_top_n_words(GoodRev['text'], 20,3)
for word, freq in common_words:
    print(word, freq)
df = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])
df.groupby('ReviewText').sum()['count'].sort_values(ascending=False).plot(
    kind='bar', title='Top 20 trigrams from good reviews')

Although there are some nonsense entries such as "http www yelp" this is starting to be helpful. We can see a few recurring themes among good reviews (e.g. "staff friendly helpful"). Nonsense entries are typically difficult to eliminate completely in NLP with user-generated text and smaller corpora. NLP is still useful *despite* the existence of nonsense results, and we should think of the output of NLP algorithms like this as a *screening tool* for finding important phrases rather than a *careful estimate* of the most important phrases. In other words, it's an application of machine intelligence to *conduct exploratory analysis* rather than to *build predictive models*.

### Question:

Look at the 5 most important bigrams for bad reviews. What *single, specific* problem seems to be the most important driver of bad reviews?

Three of the top 5 bigrams were "20 minutes", "15 minutes", and "10 minutes." These are all times, *strongly* suggesting that *waiting time for service* is a main driver for bad review scores.

### Exercise 9: (10 min)

#### 9.1

You may have noticed that many of the important "bad" bigrams included the words "like" or "just" but didn't seem very informative (e.g. "felt like", "food just"). Give some ideas of how to use this sort of observation in future pre-processing of reviews, based on the pre-processing ideas we have already studied.

**Answer.** Two potential answers are (there are many others):

1. Having recognized that these words go together in common bigrams, you could modify your algorithm so that it "clumps" these bigrams together; i.e. treats them as one word, so that your algorithm will focus on the words following that.
2. Having recognized this as a key phrase, we could have a list of the most important words that follow these key phrases (which are presumably informative). This is more time-consuming as it requires human input.

#### 9.2

Building on the previous question, we note that most of the most important complaints and compliments can't be *completely* observed by looking at bigrams or trigrams. This can often be fixed by small modifications. Do the following:

1. Write down a complaint that is unlikely to be (completely) picked up by bigram analysis. Hint: what might you write if your hamburger was served cold?
2. Write down a processing step that would fix this problem. Try to find a solution that would work for several similar problems without additional human input.

**Answer.** There are many good answers, but we focus on a simple one:

1. I would probably write "the burger was cold" or "the burger was served cold." If this were a common complaint, the most-important bigrams might include "was cold" or "served cold," which doesn't tell me *what* was served cold.
2. A simple fix, along the lines of the previous question, would be to "clump" words like "was cold" together. However, I think this is a bad fix, as it focuses too much on the word "cold" and would require a great deal of hand tuning. A better idea would be to recognize that words like "was" are always going to be a problem in this context, as they separate the important noun describing the subject ("burger") from the adjective describing the problem ("cold"). This suggests that we should take a *much* more aggressive stance towards removing stop words. This should certainly include conjugations of "to be," and likely many other common but uninformative words (like "too").

This second response is an important takeaway for NLP – this sort of problem is extremely common, and a great deal of time is often spent tweaking initial pre-processing rules. In the final part of this case, you will learn about a method that can help systematically deal with these uninformative stop words.

## Regular Expressions (25 min)

Having spent a lot of time on n-grams and how to featurize a document using them, we now take a break from `nltk` tools to introduce the most important text wrangling tool in Python (and many other languages): [**regular expressions**](https://en.wikipedia.org/wiki/Regular_expression).

The basic idea here is that you often want to perform some specific transformation (e.g. delete or substitute) every time that some possibly-complicated pattern (e.g. the letter 'A', the word 'hello', any word containing the letters 'a','r' in that order) occurs. Regular expressions are a compact and powerful language for expressing these sorts of patterns. This is super important whenever you are trying to clean a text dataset that contains thematically similar, but not exactly, the same errors. 

The terse syntax of regular expressions has led to them having a reputation for being [almost magical](https://xkcd.com/208/) in some situations (with only a few characters, you can build complete computer programs) but also for being difficult to create and read, which can [create more problems](https://xkcd.com/1171/) than they solve.

In Python, [the `re` module](https://docs.python.org/3/library/re.html) provides regular expression matching operations and common operations. Regular expressions are a deep subject, with some documentation here: https://docs.python.org/3/library/re.html?highlight=regex.

As some simple examples, we have:

1. `.` matches any character except \n (newline)
2. `\d` matches any digit (this can also be written as [0-9])
3. `\D` matches any non-digit (this can also be written as [^0-9])
4. `\w` matches any alphanumeric character ([a-zA-Z0-9_])
5. `\W` matches any non-alphanumeric character ([^a-zA-Z0-9_])

As some more complex examples, regular expressions also allow you to quantify the number of times matches can occur. For example,

1. `[a-d]+` matches any time you get $\{a,b,c,d\}$ one or more times in a row
2. `[a-d]{3}` matches any time you get them exactly 3 times in a row
3. `[a-d]*` matches any time you get them 0 or more times in a row

For now, we give a simple application based on the  `re.sub()` function, which substitutes words that match a pattern:

In [ ]:
import re
sentence = 'That was an "interesting" way to cook bread.'
pattern = r"[^\w]" # the ^ character denotes 'not', 
#                   the \w character denotes a word, and []  means
#                    anything that matches anything in the brackets. 
#                     Together, this refers to any character that is not a word.
print(re.sub(pattern, " ", sentence))

In [ ]:
str = "Natesh loves all the foold and loveds sdaslo"
x   = re.compile('lo')
iterator = x.finditer(str)
for item in iterator:
    print(item.span())
    print(item.group())

### Exercise 10: (15 min)

#### 10.1

1. Use the `re.split()` function to split the first Yelp review into a list of its constituent words.
2. Use the `re.findall()` function to search the first 30 reviews for the number of times they contain the word "food". Print the maximum number of times the word "food" is mentioned in a single review.

**Answer.** One possible solution is shown below:

In [ ]:
re.split(r'\s', AllReviews.values[0])

In [ ]:
food_count = []
for sentence in AllReviews.values:
    temp = len(re.findall('food', sentence))
    food_count.append(temp)
print(max(food_count[0:30]))

#### 10.2

Using regular expressions, find the percentage of reviews in top 500 reviews that have numbers in them.

**Answer.** One possible solution is given below:

In [ ]:
### Considering the top 500 reviews for this analysis
top_500_reviews = AllReviews.values[:500]
reviews_nos_regex = []

for each_review in top_500_reviews:
    number_list = re.findall('\d',each_review)
    
    ## number list returns all the possible digits in a review
    ## Look if the number list is empty - if so, the review has no digits in them
    if(len(number_list)) > 0:
        reviews_nos_regex.append(each_review)

In [ ]:
len(reviews_nos_regex)/len(top_500_reviews)

As is clear from above, regular expressions are very useful for extracting more general properties of text. These properties are not as informative or context-aware as n-grams can be, but they are much simpler to code and therefore can often serve as the first step of an EDA on text data.

Although regular expressions usually cannot tell us much about context overall, they *can* be used to find specific instances of words in context. For example, we may be interested in finding the first word following "good" or "bad" in a review (which can help us distinguish a positive from a negative review). Let's write some code that finds the first word following "good" in the sentence "hello I want a good burger, please.":

In [ ]:
sample = "hello I want a good burger, please"

# Find everything after "good", including "good"

post = re.findall(r'good.*', sample)[0]

print(post)

# Take just the first word after "good"

first_post = re.split(r'\s',post)[1]

print(first_post)

#### 10.3

Using the above as a template, write a generalized function that can extract the first word following "good". Don't forget to include a default behavior for when the word doesn't appear in the sentence. Run this function for all reviews and print the first 300 results for reviews that do contain the word "good".

**Answer.** One possible solution is given below:

In [ ]:
def next_word(sentence):
    post = re.findall(r'good.*', sentence)
    if (len(post) > 0):
        temp = re.split(r'\s',post[0])
        if (len(temp) > 1):
            return(temp[1])
        else:
            return('')
    else:
        return('')
    
    
print(next_word(sample))


In [ ]:
post_good = []
ind = 0
for sentence in AllReviews.values:
    temp = next_word(sentence)
    post_good.append(temp)
    
nonempty = [i for i in post_good if i] 
print(nonempty[0:300])

Skimming over the results of the previous exercise, a few things stood out:

1. People like to talk about good burgers – this appeared 5 times in the first 300 results.
2. A large number of the results are useless. One common problem is the occurrence of a sentence boundary; e.g. "good. The" near the start of the list. In this case, we should look *before* the word "good" rather than after. However, we can't look *immediately* before good – that word will usually be some conjugation of "to be", which is also not informative – rather, we need to look for a word before "good" that isn't too boring. Other times, there is a word following "good" that is uninformative; e.g. "good for" – we want to know *what* something was good for! In this case, we should keep on skimming *forward* until we see a word following "good" that isn't too boring.

In both of these cases, we can't use simple regular expressions by themselves, as regular expressions don't know how to ignore "boring" words. Regular expressions can only help us filter for the structure of words, not the content they convey within a context. We *can* use what we learned about stop words to remove these from the reviews before conducting the above analysis, but as we have been, we will still sometimes get not very informative phrases like "was cold" or "served cold". So we will introduce an alternative method, which can be applied to serve as an even better remover of stop words: **part-of-speech tagging**.

##  Part-of-speech (POS) tagging (45 min)

In English, there are eight main parts of speech - `nouns`, `pronouns`, `adjectives`, `verbs`, `adverbs`, `prepositions`, `conjunctions` and `interjections`. These are
`sustantivos`, `pronombres`, `adjetivos`, `verbos`, `adverbios`, `preposiciones`, `conjunciones` and `interjecciones`, respectively, in Spanish. The purpose of POS tagging is to label each word in a document with its part of speech.

Unsurprisingly, [POS tagging](http://www.nltk.org/book/ch05.html) can be very difficult to do by hand. `nltk` has a default function for this, called `nltk.pos_tag()`, which we will use. As a word of warning, this function is far from infallible, especially on informal text (e.g. website reviews, forum posts, text messages, etc), and words in English often exhibit POS drift (e.g. the drift of "Google" from noun to verb): 

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
#https://www.nltk.org/book/ch05.html
text_word_token = nltk.word_tokenize("Jairo is having a good day")
#text_word_token = nltk.word_tokenize(data.text[0])
nltk.pos_tag(text_word_token)
#https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

In [ ]:
I prefer to buy burgers
prefer -> burger

In [ ]:
text_word_token = nltk.word_tokenize("We are going to Race") # try "Race can be both a verb and a noun"
#text_word_token = nltk.word_tokenize(data.text[0])
nltk.pos_tag(text_word_token)

In [ ]:
#https://www.nltk.org/_modules/nltk/tag/perceptron.html
nltk.pos_tag(words)

NLTK provides documentation for each tag, which can be queried using the tag itself; e.g. `nltk.help.upenn_tagset('RB')`. Since POS is context-sensitive, POS-taggers must usually be trained on an existing corpus that has been tagged by professional linguists (possibly alongside unlabeled data to take advantage of semi-supervised methods). The most popular tag set is called the Penn Treebank set:

In [ ]:
# We can get more details about any POS tag using the help function of nltk
nltk.download('tagsets')
nltk.help.upenn_tagset('CD$')
nltk.help.upenn_tagset('NN$')

### Exercise 11: (10 min)

#### 11.1

Write code to find the percentage of reviews in the first 500 reviews of the dataset that contains a number or a cardinal using POS taggings only. (Hint: POS tag `CD` is the indicator for cardinal or number.) How does this compare to the figure we extracted from using regular expressions only?

**Answer.** One possible solution is given below:

In [ ]:
cardinal_review = []

for sentence in top_500_reviews:
    words = nltk.word_tokenize(sentence)
    cd_len = [k for k,v in nltk.pos_tag(words) if 'CD' == v]
    
    if len(cd_len) > 0:
        cardinal_review.append(sentence)

#### Proportion of reviews with a number/cardinal in it        
len(cardinal_review)/len(top_500_reviews) 
## 56.6% of the reviews have a number/cardinal in the top 500 reviews. 
## You could improve the accuracy of this estimate by looking at more than 500 reviews.

This number is considerably higher than the one we got from using regular expressions only! The reason is because POS tagging can extract numbers in text form (e.g. "one", "two") whereas regular expressions cannot. This is one advantage of using POS tagging over regular expressions.

#### 11.2

Extract all of the nouns from each review using POS tagging. This may be useful for later analysis. Even though words like "good" may be the most prevalent in good reviews, we think nouns like "service" or "burgers" are likely to be more informative.

**Answer.** One possible solution is given below:

In [ ]:
noun_reviews = []
for sentence in AllReviews.values:
    words = nltk.word_tokenize(sentence)
    noun_pt = [k for k,v in nltk.pos_tag(words) if 'NN' == v]
    noun_reviews.append(noun_pt)

### Exercise 12: (15 min)

Use POS tagging to find the first word following "good" that has an interesting POS tag. We leave this up to your discretion, but should probably include nouns and proper nouns. Inspecting the above, we think that cardinals are also almost certainly interesting: we recognize that "good 45" is probably followed by "minutes", definitely an important (though not "good") part of a review!

**Answer.** One possible solution is shown below:

In [ ]:
# First, a function that extracts only the "interesting" parts of speech

sentence = "This is a good burger, but I prefer pizza"
words = nltk.word_tokenize(sentence)
interesting = [k for k,v in nltk.pos_tag(words) if v in ['CD','FW','NN','NNS','NNP','NNPS']]

print(interesting)

In [ ]:
good_pos = ['CD','FW','NN','NNS','NNP','NNPS']

def ExtractInteresting(sentence, good):
    words = nltk.word_tokenize(sentence)
    interesting = [k for k,v in nltk.pos_tag(words) if v in good]
    return(interesting)

In [ ]:
# Next, a function that extracts the first "interesting" word that follows "good"

sentence = "This is a good burger, but I prefer pizza"
post = re.findall(r'good.*', sentence)
print(post)
# Check that post isn't empty here before doing next line
temp = ExtractInteresting(post[0],good_pos)
print(temp)
# Check that temp isn't empty here before doing next line
print(post[0])

In [ ]:
def next_word2(sentence):
    post = re.findall(r'good.*', sentence)
    if (len(post) > 0):
        temp = ExtractInteresting(post[0],good_pos)
        if (len(temp) > 0):
            return(temp[0])
        else:
            return('')
    else:
        return('')
    
    
print(next_word2(sentence))

In [ ]:
# Finally, apply this.

post_good = []
ind = 0
for sentence in AllReviews.values:
    temp = next_word2(sentence)
    post_good.append(temp)
    
nonempty = [i for i in post_good if i] 
print(nonempty[0:300])

This seems like a much more interesting list!

### Exercise 13: (10 min)

It is interesting to look specifically at Adjectives (which have a tag name of "JJ" in NLTK) when looking at reviews. We can hypothesise that good reviews and bad reviews might use very different adjectives, but that some adjectives might appear often in both good and bad reviews, as we saw with the word "good" previously.

Use POS tags to extract all adjectives from the first 500 five star reviews and the first 500 one star reviews. Extract the most 30 most commonly used adjectives from each set of reviews and print out both. Make a note of several of these; say if they appear in one or both lists, and whether or not this was expected, and why.

**Answer.** One possible solution shown below:

In [ ]:
one_star_reviews = data[data['stars']==1]['text'][:500]
five_star_reviews = data[data['stars']==5]['text'][:500]


def extract_specific_pos(reviews, pos_tag):
    results = [] 
    for review in reviews:
        words = nltk.word_tokenize(review)
        tagged = nltk.pos_tag(words)
        keep = [x[0] for x in tagged if x[1] == pos_tag]
        results += keep
    return results


negative_adjectives = extract_specific_pos(one_star_reviews, "JJ")
positive_adjectives = extract_specific_pos(five_star_reviews, "JJ")

print(Counter(negative_adjectives).most_common(30))
print(Counter(positive_adjectives).most_common(30))

As we saw before, the word "good" appears in both lists, though slightly more often in good reviews. Surprisingly, it is the most common adjective used in *bad* reviews.

Words like "disappointed", "awful", "horrible", and "rude" only appear frequently in bad reviews, as expected, while words like "favorite", "great", "fantastic", "amazing", "wonderful", and "perfect" appear in good reviews, which is also expected.

Words like "small" and "new" appear frequently in in both sets of reviews. Interestingly, so does "different", but slightly more often in good reviews, perhaps indicating that people like variety.

## Conclusions (5 min)

In this case, we focused on the basic components of an NLP pipeline, virtually all of which are frequently used *before* building a model for the business question of interest. We saw that every part of the pipeline was highly customizable, and discussed how parameters might vary depending on the specific application in mind. 

In addition to constructing a basic pipeline, we tried to give initial answers to a business question: "Which factors are most important for bad reviews?" The answers we obtained with this out-of-the-box analysis were not perfect, but they did seem to give some genuinely useful information. For example, 3 of the 5 most important phrases for bad reviews were "20 minutes", "10 minutes", and "15 minutes" – strong evidence that long service times were a major driver of bad reviews.

## Takeaways (7 min)

Text pre-processing is more complex than other forms of pre-processing you might be familiar with, as good pre-processing may rely on an enormous number of rules extracted from large corpora of English (Spanish!) text. You shouldn't try to recreate this work by yourself; instead, take advantage of large and powerful libraries such as `nltk` which have built-in corpora when possible, and use regular expressions when necessary to extend or tweak them.

Pre-processing is an extremely important and nontrivial part of NLP, and will likely take the bulk of the work for most NLP projects. Most popular parts of the pipeline come with many parameters. Yet they can give surprisingly useful summaries of entire corpora without much adjustment.

Overall, NLP can be used in many situations, but it is perhaps most useful in its ability to turn qualitative data into quantitative data. For example, if we have a collection of reviews describing, qualitatively, people’s experiences at restaurants, we can derive quantitative insights such as “X% of people who left bad reviews were unhappy with the waiting time”.

After following through this case, you know what NLP is and how it can be useful. You especially know

* The challenges of NLP: context specificity and high dimensionality.
* How to standardize and pre-process text before carrying out analysis, such as stemming 
* How to tokenize documents into sentences and words
* How to create word clouds to quickly gain high-level insights into text
* What n-grams are and how they can be created and used in analysis
* Why common (“stop”) words should often be removed before analysis
* How to find common words and n-grams
* What regular expressions are and how to use them to carry out more custom analysis
* What Parts of Speech tagging is and why analysing documents by their grammatical structure can be useful

In [ ]:
nltk, wordcloud, spacy, nagisa